In [1]:
from multiprocessing.dummy import Pool as ThreadPool
import numpy as np
import pandas as pd
import pickle
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc
import tqdm

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

import seaborn as sns
import pyswarms as ps
from matplotlib import pyplot as plt

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import zipfile

zf = zipfile.ZipFile('./data/train.csv.zip') 
train_df = pd.read_csv(zf.open('train.csv'))

y_train = train_df["TARGET"]
x_train = train_df.drop("TARGET", axis=1).values

In [4]:
classifier = LogisticRegression()
classifier.fit(x_train, y_train)
classifier.score(x_train, y_train)

0.9604314654038411

In [5]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()

def f_per_particle(m, alpha):
    total_features = 100
    
    if np.count_nonzero(m) == 0:
        x_subset = x_train
    else:
        x_subset = x_train[:,m==1]
        
    classifier.fit(x_subset, y_train)
    p = classifier.score(x_subset,y_train)

    j = (alpha * (1.0-p) + (1.0 - alpha)*(1-(x_subset.shape[1]/total_features)))
    
    return j

In [6]:
def f(x, alpha=0.88):
    n_particles = x.shape[0]
    return np.array([f_per_particle(x[i], alpha) for i in range(n_particles)])

In [8]:
options = {"c1":0.5,
          "c2":0.5,
          "w":0.9,
          "k":30,
          "p":2}

dimensions = 370
optimizer = ps.discrete.BinaryPSO(n_particles=30, dimensions=dimensions, options=options)

cost, pos = optimizer.optimize(f, iters=3)

2019-02-16 14:23:13,301 - pyswarms.discrete.binary - INFO - Optimize for 3 iters with {'c1': 0.5, 'c2': 0.5, 'w': 0.9, 'k': 30, 'p': 2}

pyswarms.discrete.binary:   0%|                                                                                    |0/3
pyswarms.discrete.binary:   0%|                                                                  |0/3, best_cost=-0.084
pyswarms.discrete.binary:  33%|██████████████████████                                            |1/3, best_cost=-0.084
pyswarms.discrete.binary:  33%|██████████████████████                                            |1/3, best_cost=-0.118
pyswarms.discrete.binary:  67%|████████████████████████████████████████████                      |2/3, best_cost=-0.118
pyswarms.discrete.binary:  67%|████████████████████████████████████████████                      |2/3, best_cost=-0.124
pyswarms.discrete.binary: 100%|██████████████████████████████████████████████████████████████████|3/3, best_cost=-0.124
2019-02-16 14:37:12,317

In [9]:
pos

array([[0, 1, 0, ..., 1, 1, 1],
       [0, 1, 0, ..., 1, 1, 1],
       [0, 1, 0, ..., 1, 1, 1],
       ...,
       [0, 1, 0, ..., 1, 1, 1],
       [0, 1, 0, ..., 1, 1, 1],
       [0, 1, 0, ..., 1, 1, 1]])

In [10]:
classifier = LogisticRegression()

x_selected_features = x_train[:, pos[0]==1]
classifier.fit(x_selected_features, y_train)
classifier.score(x_selected_features, y_train)

0.9604314654038411

In [11]:
x_selected_features.shape

(76020, 232)